In [ ]:
## Nahratie balikov, pouzitie Anaconda Jupyter notebook + updatovane baliky.
import numpy as np
import pandas as pd
import pickle

import datetime as dt
from collections import defaultdict
from tqdm import tqdm_notebook

from utils import *

In [ ]:
pocet_simulacii = 100

In [ ]:
# Nahratie populacie
pop, pop_N, N_popul = load_population()

# Nahratie OD matice
with open('../../../Dáta a analýzy/telco/arr_all.pickle','rb') as f:
    arr_all=pickle.load(f)
    f.close()
    
def load_7karty(x,arr=arr_all):
    return arr[:,:,x]

OD3 = load_7karty(2)
OD3 = OD_norm(OD3, N_popul)
locs_len = OD3.shape[0]

In [ ]:
# Priradenie populacie k obci
df_coords, data_i = load_coords('./zdroje int/obce1.xlsx', pop)

# Nahratie prvych nakaz
nakazy_sk=pd.read_excel('./zdroje int/cases_Apr16.xlsx', sheet_name = 'Hárok1')
first_infections=np.zeros(locs_len)
for i in np.arange(nakazy_sk.shape[0]):
    first_infections[pop.munic==nakazy_sk.kod_obce.iloc[i]]=nakazy_sk.pocet_pripadov.iloc[i]
    
nakazy_sk.columns=['kod_obce','obec','pocet']

In [ ]:
# Nahratie zatriedenia obci do spadovych oblasti
merges = load_merges("./zdroje int/microregions.xlsx", data_i, pop)

In [ ]:
# Definicia parametru gamma,kde 1/gamma je priemerna dlzka v kategorii I
gamma = 0.1
sigma = 1/3.1
E_start_ratio = 0.21

verzia_modelu = 'v3'
datum_cas = str(dt.datetime.now())[0:10] + ' ' + str(dt.datetime.now())[11:13] + 'h' + str(dt.datetime.now())[14:16] + 'm'

In [ ]:
# Nahratie vekovej struktury obce
data_age=pd.read_excel('./zdroje ext/vekove_zlozenie_obec.xlsx')
data_age=data_age.sort_values(['id'])

# Izolacia 70+, 70%
N_popul_senior_izol=remove_age(N_popul,70,120,0.7,data=data_age)

In [ ]:
# Pridanie marginalizovanych romskych komunit s poctom MRK > 200 ako dalsich obci/spadovych oblasti pre moznost priameho modelovania
OD_MRK, R0_correction_demogr_mrk,N_popul_mrk, base_munic ,mrk_munic_list = mrk_matrix_change('./zdroje int/KoeficientyR0_v2.xlsx', pop, pop_N, data_i, OD3, N_popul, alfa_to_mrk=1/5, alfa_from_mrk=1/50)
# MRK izolacia
OD_MRK1, R0_correction_demogr_mrk, N_popul_mrk, base_munic ,mrk_munic_list = mrk_matrix_change('./zdroje int/KoeficientyR0_v2.xlsx', pop, pop_N, data_i, OD3, N_popul, alfa_to_mrk=1/15, alfa_from_mrk=1/150)
OD_MRK2, R0_correction_demogr_mrk, N_popul_mrk, base_munic ,mrk_munic_list = mrk_matrix_change('./zdroje int/KoeficientyR0_v2.xlsx', pop, pop_N, data_i, OD3, N_popul, alfa_to_mrk=1/45, alfa_from_mrk=1/350)
OD_MRK3, R0_correction_demogr_mrk, N_popul_mrk, base_munic ,mrk_munic_list = mrk_matrix_change('./zdroje int/KoeficientyR0_v2.xlsx', pop, pop_N, data_i, OD3, N_popul, alfa_to_mrk=1/200, alfa_from_mrk=1/1000)

# Seniori izolacia
OD_MRK_si, R0_correction_demogr_mrk_si, N_popul_mrk_si, base_munic_si ,mrk_munic_list = mrk_matrix_change('./zdroje int/KoeficientyR0_v2.xlsx', pop, pop_N, data_i, OD3, N_popul_senior_izol, alfa_to_mrk=1/5, alfa_from_mrk=1/50)
first_infections_mrk = np.pad(first_infections, (0, OD_MRK.shape[0] - first_infections.shape[0]),mode='constant')

In [ ]:
# Mapovanie povodnych ID (obce) na nove ID (spadove oblasti)
new_ids = prepare_merge(merges, len(N_popul_mrk), 1)
n_locs_new = max(new_ids) + 1
OD_merged = np.zeros((n_locs_new, n_locs_new))
OD_merged_mrk1 = np.zeros((n_locs_new, n_locs_new))
OD_merged_mrk2 = np.zeros((n_locs_new, n_locs_new))
OD_merged_mrk3 = np.zeros((n_locs_new, n_locs_new))

N_popul_merged = np.zeros(n_locs_new)
N_popul_merged_si = np.zeros(n_locs_new)
R0_correction_demogr_merged = np.ones(n_locs_new)

for i in tqdm_notebook(range(OD_MRK.shape[0])):
    for j in range(OD_MRK.shape[0]):
        OD_merged[new_ids[i]][new_ids[j]] += OD_MRK[i][j]
        OD_merged_mrk1[new_ids[i]][new_ids[j]] += OD_MRK1[i][j]
        OD_merged_mrk2[new_ids[i]][new_ids[j]] += OD_MRK2[i][j]
        OD_merged_mrk3[new_ids[i]][new_ids[j]] += OD_MRK3[i][j]

    N_popul_merged[new_ids[i]] += N_popul_mrk[i]
    N_popul_merged_si[new_ids[i]] += N_popul_mrk_si[i]
    R0_correction_demogr_merged[new_ids[i]] = max(R0_correction_demogr_merged[new_ids[i]], R0_correction_demogr_mrk[i])
    
OD_merged_norm = OD_norm(OD_merged, N_popul_merged)
OD_merged_norm_mrk1 = OD_norm(OD_merged_mrk1, N_popul_merged)
OD_merged_norm_mrk2 = OD_norm(OD_merged_mrk2, N_popul_merged)
OD_merged_norm_mrk3 = OD_norm(OD_merged_mrk3, N_popul_merged)
OD_merged_norm_si = OD_norm(OD_merged, N_popul_merged_si)

In [ ]:
## Zaciatocna difuzia infekcii podla OD matice
first_infections_merged = initiation_diffusion_no_age(pop,nakazy_sk, new_ids, arr_all)

In [ ]:
infect_mrk = pd.read_csv('./zdroje int/infect_mrk.csv', sep = ';')

In [ ]:
for i in range(infect_mrk.shape[0]):
    k=np.where(infect_mrk.iloc[i,0]==np.array(mrk_munic_list))[0]
    first_infections_merged[-261:][k]=infect_mrk.iloc[i,1]*3

In [ ]:
## Modelovanie prichodu infikovanych osob zo zahranicia
with open('../../../Dáta a analýzy/telco/returners.pickle','rb') as a:                                                           
    returners=pickle.load(a)                                                                         
    a.close()                                                                                        

return_vec=(returners['high_risk'] + returners['low_risk'])                                            
return_vec=np.concatenate([((np.tile([1.0],2926)) *( np.repeat(return_vec,1)) ),(np.repeat(0,261*1))])
return_vec_micro=np.zeros(np.unique(new_ids).shape[0])                                        
for i in np.unique(new_ids):                                                                  
    return_vec_micro[i]=return_vec[(np.array(new_ids)==i)].sum() 

In [ ]:
## Pouzitie mocninovej distribucie pre betu
beta_list_240, beta_vec_izol, beta_vec_neizol = prepare_beta_list(U = 0.7, k_min = 0.165, k_max = 20, kappa = 2.5, gamma_factor=30, izol_beta_base=0.15)

# Vytvorenie bety pre dalsie priemerne hodnoty mocninovej distribucie
beta_list_220 = adjust_beta(beta_new = 0.220, beta_orig=beta_list_240)
beta_list_160 = adjust_beta(beta_new = 0.160, beta_orig=beta_list_240)
beta_list_110 = adjust_beta(beta_new = 0.110, beta_orig=beta_list_240)
beta_list_80  = adjust_beta(beta_new = 0.080, beta_orig=beta_list_240)

In [ ]:
# Simulacia hlavne scenare, vstupy
pocet_scenarov = 3
total_days = 600

# Alpha zostava 1, vyuzite data o realnych presunoch od operatorov
alpha_vec = [np.repeat(1.0, total_days) for _ in range(pocet_scenarov)]
# Parametre pre betu
beta_list = [beta_list_110, beta_list_80, beta_list_80]
# Beta zostava staticka
beta_mult = [np.ones(total_days) for _ in range(pocet_scenarov)]
# Zohladnenie socio-ekonomickych charakteristik
R0_correction_demogr_list=[R0_correction_demogr_merged,R0_correction_demogr_merged,np.ones(R0_correction_demogr_merged.shape[0])]
# Vyuzitie najnovsich dat od operatorov vo vsetkych scenaroch
OD_vec = [OD_merged_norm for _ in range(pocet_scenarov)]

# Sumarne info pre kazdy scenar
for scen in range(pocet_scenarov):
    print('Scenar:', str(scen),
          'Avg alpha: ', np.round(np.average(alpha_vec[scen]),3),
          'Avg R0:', np.round(np.average(np.mean(beta_list[scen])/gamma),3),
          'Avg beta_mult:', np.round(np.average(beta_mult[scen]),3),
          'Avg R0 correction demogr:', np.round(np.average(R0_correction_demogr_list[scen]), 3))

dict_tmp = {}
m = 0
while m < pocet_scenarov:
    key1 = 'SIR_scenar' + str(m)
    key2 = 'obce_scenar' + str(m)
    dict_tmp[key1] = []
    dict_tmp[key2] = []
    m += 1

for sim in range(pocet_simulacii):
    print('Simulacia:'+str(sim))
    for scen in range(pocet_scenarov):
        key1 = 'SIR_scenar' + str(scen)
        key2 = 'obce_scenar' + str(scen)
        SIR,obce =simulation_SEIR(locs_len=OD_vec[scen].shape[0],N_popul=N_popul_merged, OD=OD_vec[scen],alpha_vec=alpha_vec[scen],
                      beta_mult=beta_mult[scen],R0_correction_demogr=R0_correction_demogr_list[scen],
                      first_infections=first_infections_merged,beta_list=beta_list[scen],gamma=gamma,tau= 16/24, T = total_days,
                      new_infect_rounding_fn=prob_rounding,
                      return_vec_micro=return_vec_micro,
                      sigma=sigma, E_start_ratio=E_start_ratio)
       
        dict_tmp[key1].append(SIR)
        dict_tmp[key2].append(obce)
        
## Spriemerovanie simulacii    
dictagg = {}
m = 0
while m < pocet_scenarov:
    key1 = 'SIR_scenar' + str(m)
    key2 = 'obce_scenar' + str(m)
    dictagg[key1] = []
    dictagg[key2] = []
    m += 1

for m in range(pocet_scenarov):
    key1 = 'SIR_scenar' + str(m)
    key2 = 'obce_scenar' + str(m)
    
    dictagg[key1] = pd.DataFrame({
                               'dni':np.arange(0,total_days+1),
                               'sus': np.array(meanlist_sus(dict_tmp[key1]), dtype=np.float64),
                               'exp': np.array(meanlist_exp(dict_tmp[key1]), dtype=np.float64),
                               'inf': np.array(meanlist_inf(dict_tmp[key1]), dtype=np.float64),
                               'inf5': np.array(perclist_inf(dict_tmp[key1], 5), dtype=np.float64),
                               'inf95': np.array(perclist_inf(dict_tmp[key1], 95), dtype=np.float64),
                               'exp5': np.array(perclist_exp(dict_tmp[key1], 5), dtype=np.float64),
                               'exp95': np.array(perclist_exp(dict_tmp[key1], 95), dtype=np.float64),
                               'rec': np.array(meanlist_rec(dict_tmp[key1]), dtype=np.float64),
                               'std_inf': np.array(stdlist_inf(dict_tmp[key1]), dtype=np.float64)})     

    dictagg[key2] = [np.arange(1,total_days+1), sumlist(dict_tmp[key2])[:,0,:], sumlist(dict_tmp[key2])[:,1,:], sumlist(dict_tmp[key2])[:,2,:], sumlist(dict_tmp[key2])[:,3,:]]

path_hlavne = './vystupy_model/simulations_' + verzia_modelu + '_' + datum_cas + '_' + 'hlavne_scenare.pickle' 
with open(path_hlavne,'wb') as f:
    pickle.dump(dictagg,f)

In [ ]:
## Simulacia vstupy, lockdown
pocet_scenarov = 3
total_days = 600

alpha_vec = [np.ones(600), np.concatenate([np.ones(21)/10,
                                           np.ones(600-21)]),
             np.concatenate([np.ones(42)/10,np.ones(600-42)])]
beta_list = [beta_list_80, beta_list_80, beta_list_80]

l = 0.3/1.1
beta_mult = [np.ones(600), np.concatenate([np.ones(21)*l,np.ones(600-21)]),np.concatenate([np.ones(42)*l,np.ones(600-42)])]
R0_correction_demogr_list=[R0_correction_demogr_merged, R0_correction_demogr_merged,R0_correction_demogr_merged]
OD_vec = [OD_merged_norm for _ in range(pocet_scenarov)]
for scen in range(pocet_scenarov):
    print('Scenar:', str(scen),
          'Avg alpha: ', np.round(np.average(alpha_vec[scen]),3),
          'Avg R0:', np.round(np.average(np.mean(beta_list[scen])/gamma),3),
          'Avg beta_mult:', np.round(np.average(beta_mult[scen]),3))

dict_tmp = {}
m = 0
while m < pocet_scenarov:
    key1 = 'SIR_scenar' + str(m)
    key2 = 'obce_scenar' + str(m)
    dict_tmp[key1] = []
    dict_tmp[key2] = []
    m += 1

for sim in range(pocet_simulacii):
    print('Simulacia:'+str(sim))
    for scen in range(pocet_scenarov):
        key1 = 'SIR_scenar' + str(scen)
        key2 = 'obce_scenar' + str(scen)
        SIR,obce =simulation_SEIR(locs_len=OD_vec[scen].shape[0],N_popul=N_popul_merged, OD=OD_vec[scen],alpha_vec=alpha_vec[scen],
                      beta_mult=beta_mult[scen],R0_correction_demogr=R0_correction_demogr_list[scen],
                      first_infections=first_infections_merged,beta_list=beta_list[scen],gamma=gamma,tau= 16/24, T = total_days,
                      new_infect_rounding_fn=prob_rounding,
                      return_vec_micro=return_vec_micro,
                      sigma=sigma, E_start_ratio=E_start_ratio)      

        dict_tmp[key1].append(SIR)
        dict_tmp[key2].append(obce)
        
## Spriemerovanie simulacii    
dictagg = {}
m = 0
while m < pocet_scenarov:
    key1 = 'SIR_scenar' + str(m)
    key2 = 'obce_scenar' + str(m)
    dictagg[key1] = []
    dictagg[key2] = []
    m += 1

for m in range(pocet_scenarov):
    key1 = 'SIR_scenar' + str(m)
    key2 = 'obce_scenar' + str(m)
    
    dictagg[key1] = pd.DataFrame({
                               'dni':np.arange(0,total_days+1),
                               'sus': np.array(meanlist_sus(dict_tmp[key1]), dtype=np.float64),
                               'exp': np.array(meanlist_exp(dict_tmp[key1]), dtype=np.float64),
                               'inf': np.array(meanlist_inf(dict_tmp[key1]), dtype=np.float64),
                               'inf5': np.array(perclist_inf(dict_tmp[key1], 5), dtype=np.float64),
                               'inf95': np.array(perclist_inf(dict_tmp[key1], 95), dtype=np.float64),
                               'exp5': np.array(perclist_exp(dict_tmp[key1], 5), dtype=np.float64),
                               'exp95': np.array(perclist_exp(dict_tmp[key1], 95), dtype=np.float64),
                               'rec': np.array(meanlist_rec(dict_tmp[key1]), dtype=np.float64),
                               'std_inf': np.array(stdlist_inf(dict_tmp[key1]), dtype=np.float64)})     

    dictagg[key2] = [np.arange(1,total_days+1), sumlist(dict_tmp[key2])[:,0,:], sumlist(dict_tmp[key2])[:,1,:], sumlist(dict_tmp[key2])[:,2,:]]

path_lockdown = './vystupy_model/simulations_' + verzia_modelu + '_' + datum_cas+ '_' + 'lockdown.pickle' 
with open(path_lockdown,'wb') as f:
    pickle.dump(dictagg,f)

In [ ]:
# Simulacia izolacia 70+, 70%
pocet_scenarov = 2
total_days = 600

# Alpha zostava 1, vyuzite data o realnych presunoch od operatorov
alpha_vec = [np.repeat(1.0, total_days) for _ in range(pocet_scenarov)]
# Parametre pre betu
beta_list = [beta_list_80, beta_list_80]
# Beta zostava staticka
beta_mult = [np.ones(total_days) for _ in range(pocet_scenarov)]
# Zohladnenie socio-ekonomickych charakteristik
R0_correction_demogr_list=[R0_correction_demogr_merged,R0_correction_demogr_merged]
# Vyuzitie najnovsich dat od operatorov vo vsetkych scenaroch
OD_vec = [OD_merged_norm, OD_merged_norm_si]
# Populacny vektor komplet a s izolaciou seniorov
N_popul_vec = [N_popul_merged, N_popul_merged]
N_popul_new_vec = [N_popul_merged, N_popul_merged_si]

# Sumarne info pre kazdy scenar
for scen in range(pocet_scenarov):
    print('Scenar:', str(scen),
          'Avg alpha: ', np.round(np.average(alpha_vec[scen]),3),
          'Avg R0:', np.round(np.average(np.mean(beta_list[scen])/gamma),3),
          'Avg beta_mult:', np.round(np.average(beta_mult[scen]),3))

dict_tmp = {}
m = 0
while m < pocet_scenarov:
    key1 = 'SIR_scenar' + str(m)
    key2 = 'obce_scenar' + str(m)
    dict_tmp[key1] = []
    dict_tmp[key2] = []
    m += 1

for sim in range(pocet_simulacii):
    print('Simulacia:'+str(sim))
    for scen in range(pocet_scenarov):
        key1 = 'SIR_scenar' + str(scen)
        key2 = 'obce_scenar' + str(scen)
        SIR,obce =simulation_SEIR_remove_age(locs_len=OD_vec[scen].shape[0],N_popul=N_popul_vec[scen], 
                      N_popul_new = N_popul_new_vec[scen], OD=OD_vec[scen],alpha_vec=alpha_vec[scen],
                      beta_mult=beta_mult[scen],R0_correction_demogr=R0_correction_demogr_list[scen],
                      first_infections=first_infections_merged,beta_list=beta_list[scen],gamma=gamma,tau= 16/24, T = total_days,
                      new_infect_rounding_fn=prob_rounding,
                      return_vec_micro=return_vec_micro,
                      sigma=sigma, E_start_ratio=E_start_ratio)
       
        dict_tmp[key1].append(SIR)
        dict_tmp[key2].append(obce)
        
## Spriemerovanie simulacii    
dictagg = {}
m = 0
while m < pocet_scenarov:
    key1 = 'SIR_scenar' + str(m)
    key2 = 'obce_scenar' + str(m)
    dictagg[key1] = []
    dictagg[key2] = []
    m += 1

for m in range(pocet_scenarov):
    key1 = 'SIR_scenar' + str(m)
    key2 = 'obce_scenar' + str(m)
    
    dictagg[key1] = pd.DataFrame({
                               'dni':np.arange(0,total_days+1),
                               'sus': np.array(meanlist_sus(dict_tmp[key1]), dtype=np.float64),
                               'exp': np.array(meanlist_exp(dict_tmp[key1]), dtype=np.float64),
                               'inf': np.array(meanlist_inf(dict_tmp[key1]), dtype=np.float64),
                               'inf5': np.array(perclist_inf(dict_tmp[key1], 5), dtype=np.float64),
                               'inf95': np.array(perclist_inf(dict_tmp[key1], 95), dtype=np.float64),
                               'exp5': np.array(perclist_exp(dict_tmp[key1], 5), dtype=np.float64),
                               'exp95': np.array(perclist_exp(dict_tmp[key1], 95), dtype=np.float64),
                               'rec': np.array(meanlist_rec(dict_tmp[key1]), dtype=np.float64),
                               'std_inf': np.array(stdlist_inf(dict_tmp[key1]), dtype=np.float64)})     

    dictagg[key2] = [np.arange(1,total_days+1), sumlist(dict_tmp[key2])[:,0,:], sumlist(dict_tmp[key2])[:,1,:], sumlist(dict_tmp[key2])[:,2,:]]

path_senior = './vystupy_model/simulations_' + verzia_modelu + '_' + datum_cas+ '_' + 'senior.pickle' 
with open(path_senior,'wb') as f:
    pickle.dump(dictagg,f)

In [ ]:
## Simulacia MRK level mobility
pocet_scenarov = 4
total_days = 600

alpha_vec = [np.ones(600) for _ in range(pocet_scenarov)]
beta_list = [beta_list_80 for _ in range(pocet_scenarov)]
beta_mult = [np.ones(600) for _ in range(pocet_scenarov)]

R0_correction_demogr_list=[R0_correction_demogr_merged for _ in range(pocet_scenarov)]
OD_vec = [OD_merged_norm,OD_merged_norm_mrk1,
          OD_merged_norm_mrk2,OD_merged_norm_mrk3]

for scen in range(pocet_scenarov):
    print('Scenar:', str(scen),
          'Avg alpha: ', np.round(np.average(alpha_vec[scen]),3),
          'Avg R0:', np.round(np.average(np.mean(beta_list[scen])/gamma),3),
          'Avg beta_mult:', np.round(np.average(beta_mult[scen]),3))

dict_tmp = {}
m = 0
while m < pocet_scenarov:
    key1 = 'SIR_scenar' + str(m)
    key2 = 'obce_scenar' + str(m)
    dict_tmp[key1] = []
    dict_tmp[key2] = []
    m += 1

for sim in range(pocet_simulacii):
    print('Simulacia:'+str(sim))
    for scen in range(pocet_scenarov):
        key1 = 'SIR_scenar' + str(scen)
        key2 = 'obce_scenar' + str(scen)
        SIR,obce =simulation_SEIR(locs_len=OD_vec[scen].shape[0],N_popul=N_popul_merged, OD=OD_vec[scen],alpha_vec=alpha_vec[scen],
                      beta_mult=beta_mult[scen],R0_correction_demogr=R0_correction_demogr_list[scen],
                      first_infections=first_infections_merged,beta_list=beta_list[scen],gamma=gamma,tau= 16/24, T = total_days,
                      new_infect_rounding_fn=prob_rounding,
                      return_vec_micro=return_vec_micro,
                      sigma=sigma, E_start_ratio=E_start_ratio)        

        dict_tmp[key1].append(SIR)
        dict_tmp[key2].append(obce)
        
## Spriemerovanie simulacii    
dictagg = {}
m = 0
while m < pocet_scenarov:
    key1 = 'SIR_scenar' + str(m)
    key2 = 'obce_scenar' + str(m)
    dictagg[key1] = []
    dictagg[key2] = []
    m += 1

for m in range(pocet_scenarov):
    key1 = 'SIR_scenar' + str(m)
    key2 = 'obce_scenar' + str(m)
    
    dictagg[key1] = pd.DataFrame({
                               'dni':np.arange(0,total_days+1),
                               'sus': np.array(meanlist_sus(dict_tmp[key1]), dtype=np.float64),
                               'exp': np.array(meanlist_exp(dict_tmp[key1]), dtype=np.float64),
                               'inf': np.array(meanlist_inf(dict_tmp[key1]), dtype=np.float64),
                               'inf5': np.array(perclist_inf(dict_tmp[key1], 5), dtype=np.float64),
                               'inf95': np.array(perclist_inf(dict_tmp[key1], 95), dtype=np.float64),
                               'exp5': np.array(perclist_exp(dict_tmp[key1], 5), dtype=np.float64),
                               'exp95': np.array(perclist_exp(dict_tmp[key1], 95), dtype=np.float64),
                               'rec': np.array(meanlist_rec(dict_tmp[key1]), dtype=np.float64),
                               'std_inf': np.array(stdlist_inf(dict_tmp[key1]), dtype=np.float64)})     

    dictagg[key2] = [np.arange(1,total_days+1), sumlist(dict_tmp[key2])[:,0,:], sumlist(dict_tmp[key2])[:,1,:], sumlist(dict_tmp[key2])[:,2,:]]

path_MRK_mobilita = './vystupy_model/simulations_' + verzia_modelu + '_' + datum_cas+ '_' + 'MRK_mobilita.pickle' 
with open(path_MRK_mobilita,'wb') as f:
    pickle.dump(dictagg,f)

In [ ]:
# Simulacia priloha B, kratsia doba infekcnosti a z toho vyplyvajuca vyssia beta
pocet_scenarov = 3
total_days = 600
gamma = 0.2

# Alpha zostava 1, vyuzite data o realnych presunoch od operatorov
alpha_vec = [np.repeat(1.0, total_days) for _ in range(pocet_scenarov)]
# Parametre pre betu
beta_list = [beta_list_220, beta_list_160, beta_list_160]
# Beta zostava staticka
beta_mult = [np.ones(total_days) for _ in range(pocet_scenarov)]
# Zohladnenie socio-ekonomickych charakteristik
R0_correction_demogr_list=[R0_correction_demogr_merged,R0_correction_demogr_merged,np.ones(R0_correction_demogr_merged.shape[0])]
# Vyuzitie najnovsich dat od operatorov vo vsetkych scenaroch
OD_vec = [OD_merged_norm for _ in range(pocet_scenarov)]

# Sumarne info pre kazdy scenar
for scen in range(pocet_scenarov):
    print('Scenar:', str(scen),
          'Avg alpha: ', np.round(np.average(alpha_vec[scen]),3),
          'Avg R0:', np.round(np.average(np.mean(beta_list[scen])/gamma),3),
          'Avg beta_mult:', np.round(np.average(beta_mult[scen]),3))

dict_tmp = {}
m = 0
while m < pocet_scenarov:
    key1 = 'SIR_scenar' + str(m)
    key2 = 'obce_scenar' + str(m)
    dict_tmp[key1] = []
    dict_tmp[key2] = []
    m += 1

for sim in range(pocet_simulacii):
    print('Simulacia:'+str(sim))
    for scen in range(pocet_scenarov):
        key1 = 'SIR_scenar' + str(scen)
        key2 = 'obce_scenar' + str(scen)
        SIR,obce =simulation_SEIR(locs_len=OD_vec[scen].shape[0],N_popul=N_popul_merged, OD=OD_vec[scen],alpha_vec=alpha_vec[scen],
                      beta_mult=beta_mult[scen],R0_correction_demogr=R0_correction_demogr_list[scen],
                      first_infections=first_infections_merged,beta_list=beta_list[scen],gamma=gamma,tau= 16/24, T = total_days,
                      new_infect_rounding_fn=prob_rounding,
                      return_vec_micro=return_vec_micro,
                      sigma=sigma, E_start_ratio=E_start_ratio)
       
        dict_tmp[key1].append(SIR)
        dict_tmp[key2].append(obce)
        
## Spriemerovanie simulacii    
dictagg = {}
m = 0
while m < pocet_scenarov:
    key1 = 'SIR_scenar' + str(m)
    key2 = 'obce_scenar' + str(m)
    dictagg[key1] = []
    dictagg[key2] = []
    m += 1

for m in range(pocet_scenarov):
    key1 = 'SIR_scenar' + str(m)
    key2 = 'obce_scenar' + str(m)
    
    dictagg[key1] = pd.DataFrame({
                               'dni':np.arange(0,total_days+1),
                               'sus': np.array(meanlist_sus(dict_tmp[key1]), dtype=np.float64),
                               'exp': np.array(meanlist_exp(dict_tmp[key1]), dtype=np.float64),
                               'inf': np.array(meanlist_inf(dict_tmp[key1]), dtype=np.float64),
                               'inf5': np.array(perclist_inf(dict_tmp[key1], 5), dtype=np.float64),
                               'inf95': np.array(perclist_inf(dict_tmp[key1], 95), dtype=np.float64),
                               'exp5': np.array(perclist_exp(dict_tmp[key1], 5), dtype=np.float64),
                               'exp95': np.array(perclist_exp(dict_tmp[key1], 95), dtype=np.float64),
                               'rec': np.array(meanlist_rec(dict_tmp[key1]), dtype=np.float64),
                               'std_inf': np.array(stdlist_inf(dict_tmp[key1]), dtype=np.float64)})     

    dictagg[key2] = [np.arange(1,total_days+1), sumlist(dict_tmp[key2])[:,0,:], sumlist(dict_tmp[key2])[:,1,:], sumlist(dict_tmp[key2])[:,2,:], sumlist(dict_tmp[key2])[:,3,:]]

path_priloha = './vystupy_model/simulations_' + verzia_modelu + '_' + datum_cas+ '_' + 'gamma_0.2.pickle' 
with open(path_priloha,'wb') as f:
    pickle.dump(dictagg,f)

In [ ]:
dictagg['SIR_scenar0'].to_excel('./betadecay_lag30.xlsx')
dictagg['SIR_scenar1'].to_excel('./betadecay_lag35.xlsx')
dictagg['SIR_scenar2'].to_excel('./betadecay_lag40.xlsx')

## Grafy

### Hlavne scenare

In [ ]:
verzia_modelu = 'v3'
datum_cas = '2020-04-21 17h45m'

path_hlavne = './vystupy_model/simulations_' + verzia_modelu + '_' + datum_cas + '_' + 'hlavne_scenare.pickle'
path_lockdown = './vystupy_model/simulations_' + verzia_modelu + '_' + datum_cas+ '_' + 'lockdown.pickle'
path_senior = './vystupy_model/simulations_' + verzia_modelu + '_' + datum_cas+ '_' + 'senior.pickle'
path_MRK_mobilita = './vystupy_model/simulations_' + verzia_modelu + '_' + datum_cas+ '_' + 'MRK_mobilita.pickle'
path_priloha = './vystupy_model/simulations_' + verzia_modelu + '_' + datum_cas+ '_' + 'gamma_0.2.pickle'

In [ ]:
with open(path_hlavne,'rb') as f:
    dictagg=pickle.load(f)
    f.close()

In [ ]:
total_days = 600
scen_vyber = [0,1,2]
title = 'Porovnanie scenárov šírenia nákazy'
legenda_vsetky_scenare = ['Scenár1: R0 1.10','Scenár2: R0 0.80','Scenár3: R0 0.80, socio-demog. neutral']

name = verzia_modelu + '_' + 'hlavne_scenare' + '_' + datum_cas
plot_peaks(dictagg = dictagg, scen_vyber = scen_vyber, legenda_vsetky_scenare = legenda_vsetky_scenare,
           verzia_modelu = name, title = title, total_days = total_days, SEIR = False)

In [ ]:
with open(path_hlavne,'rb') as f:
    dictagg=pickle.load(f)
    f.close()
    
dictagg['SIR_scenar0'].loc[:,['dni','inf','inf5','inf95']].to_excel('./export pre novinarov/Hlavne_Scenar1.xlsx', index=False)
dictagg['SIR_scenar1'].loc[:,['dni','inf','inf5','inf95']].to_excel('./export pre novinarov/Hlavne_Scenar2_default.xlsx', index=False)
dictagg['SIR_scenar2'].loc[:,['dni','inf','inf5','inf95']].to_excel('./export pre novinarov/Hlavne_Scenar3.xlsx', index=False)

dictagg['SIR_scenar1'].loc[:,['dni','sus','exp','inf','rec']].to_excel('./IFP_Scenar2_default.xlsx', index=False)

### Lockdown

In [ ]:
with open(path_lockdown,'rb') as f:
    dictagg=pickle.load(f)
    f.close()

total_days = 600
scen_vyber = [0,1,2]
legenda_vsetky_scenare = ['Scenár1: R0 0.80','Scenár2: Lockdown 3 týždne, potom R0 0.80', 'Scenár3: Lockdown 6 týždňov, potom R0 0.80']

title = 'Model lockdownu na úrovni 3 a 6 týždňov'

name = verzia_modelu + '_' + 'lockdown' + '_' + datum_cas
plot_peaks_lockdown(dictagg = dictagg, scen_vyber = scen_vyber, legenda_vsetky_scenare = legenda_vsetky_scenare, 
                    verzia_modelu = name, total_days = total_days, title = title, SEIR = False)

In [ ]:
with open(path_lockdown,'rb') as f:
    dictagg=pickle.load(f)
    f.close()

dictagg['SIR_scenar0'].loc[:,['dni','inf','inf5','inf95']].to_excel('./export pre novinarov/Lockdown_Scenar1.xlsx', index=False)
dictagg['SIR_scenar1'].loc[:,['dni','inf','inf5','inf95']].to_excel('./export pre novinarov/Lockdown_Scenar2.xlsx', index=False)
dictagg['SIR_scenar2'].loc[:,['dni','inf','inf5','inf95']].to_excel('./export pre novinarov/Lockdown_Scenar3.xlsx', index=False)

### Izolacia seniorov

In [ ]:
with open(path_senior,'rb') as f:
    dictagg=pickle.load(f)
    f.close()

In [ ]:
total_days = 600
scen_vyber = [0,1]
title = 'Porovnanie scenárov šírenia nákazy'
legenda_vsetky_scenare = ['Scenár1: R0 0.80', 'Scenár2: R0 0.80, izolácia seniorov']

name = verzia_modelu + '_' + 'seniori' + '_' + datum_cas
plot_peaks(dictagg = dictagg, scen_vyber = scen_vyber, legenda_vsetky_scenare = legenda_vsetky_scenare,
           verzia_modelu = name, title = title, total_days = total_days, SEIR = False)

In [ ]:
with open(path_senior,'rb') as f:
    dictagg=pickle.load(f)
    f.close()

dictagg['SIR_scenar0'].loc[:,['dni','inf','inf5','inf95']].to_excel('./export pre novinarov/Seniori_Scenar1.xlsx', index=False)
dictagg['SIR_scenar1'].loc[:,['dni','inf','inf5','inf95']].to_excel('./export pre novinarov/Seniori_Scenar2.xlsx', index=False)

### Intenzita kontaktu MRK

In [ ]:
with open(path_MRK_mobilita,'rb') as f:
    dictagg=pickle.load(f)
    f.close()

In [ ]:
total_days = 600
scen_vyber = [0,1,2]
title = 'Porovnanie scenárov šírenia nákazy'
legenda_vsetky_scenare = ['Scenár1: R0 0.80','Scenár2: R0 0.80 MRK nízka izolácia',
                          'Scenár3: R0 0.80 MRK vysoká izolácia','Scenár4: R0 0.80 MRK total isolation']

name = verzia_modelu + '_' + 'MRK_alpha_mobilita' + '_' + datum_cas
plot_peaks(dictagg = dictagg, scen_vyber = scen_vyber, legenda_vsetky_scenare = legenda_vsetky_scenare,
           verzia_modelu = name, title = title, total_days = total_days, SEIR = False)

In [ ]:
with open(path_MRK_mobilita,'rb') as f:
    dictagg=pickle.load(f)
    f.close()

dictagg['SIR_scenar0'].loc[:,['dni','inf','inf5','inf95']].to_excel('./export pre novinarov/MRK_Scenar1.xlsx', index=False)
dictagg['SIR_scenar1'].loc[:,['dni','inf','inf5','inf95']].to_excel('./export pre novinarov/MRK_Scenar2.xlsx', index=False)
dictagg['SIR_scenar2'].loc[:,['dni','inf','inf5','inf95']].to_excel('./export pre novinarov/MRK_Scenar3.xlsx', index=False)

### Graf príloha B, gamma 0.2

In [ ]:
with open(path_priloha,'rb') as f:
    dictagg=pickle.load(f)
    f.close()

In [ ]:
total_days = 600
scen_vyber = [0,1,2]
title = 'Porovnanie scenárov šírenia nákazy'
legenda_vsetky_scenare = ['Scenár1: R0 1.10', 'Scenár2: R0 0.80', 
                          'Scenár3: R0 0.80, socio-demog. neutral']

name = verzia_modelu + '_' + 'priloha_B' + '_' + datum_cas
plot_peaks(dictagg = dictagg, scen_vyber = scen_vyber, legenda_vsetky_scenare = legenda_vsetky_scenare,
           verzia_modelu = name, title = title, total_days = total_days, SEIR = False)

In [ ]:
dictagg['SIR_scenar0'].loc[:,['dni','inf','inf5','inf95']].to_excel('./export pre novinarov/Priloha_B_Scenar1.xlsx', index=False)
dictagg['SIR_scenar1'].loc[:,['dni','inf','inf5','inf95']].to_excel('./export pre novinarov/Priloha_B_Scenar2.xlsx', index=False)
dictagg['SIR_scenar2'].loc[:,['dni','inf','inf5','inf95']].to_excel('./export pre novinarov/Priloha_B_Scenar3.xlsx', index=False)

## Tabuľky

In [ ]:
with open(path_hlavne,'rb') as f:
    dictagg=pickle.load(f)
    f.close()
    
N = pop.popul.sum()

In [ ]:
## Tabulky 1, 3
scen_vyber = [0,1,2]
    
## Tabulka 1
total_days = 600

A = pd.DataFrame({'Dni':dictagg['SIR_scenar' + str(scen_vyber[0])].dni,
              'I_high':dictagg['SIR_scenar' + str(scen_vyber[0])].inf*N,
              'I_mid':dictagg['SIR_scenar' + str(scen_vyber[1])].inf*N,
              'I_low':dictagg['SIR_scenar' + str(scen_vyber[2])].inf*N})  

A.columns = ['Dni', 'R0 1.1, Počet infikovaných','R0 0.8, Počet infikovaných',
             'R0 0.80 socio-demog. neutral, Počet infikovaných'] 

# Vyber konkretnych dni
A = A.iloc[ [10,20,30,50,80,100,150,200,250] , :]

path = './tabulky/' + verzia_modelu + '_' + 'Tabulka1' + '_' + datum_cas + '.xlsx'
A.to_excel(path, engine='xlsxwriter', index = False)

In [ ]:
with open(path_hlavne,'rb') as f:
    dictagg=pickle.load(f)
    f.close()
    
## Vystupna bohatsia tabulka pre vybrany scenar
scen_vyber = 1
gamma = 0.1

scen = 'obce_scenar' + str(scen_vyber)
N = pop.popul.sum()
df = pd.DataFrame({'Dni':dictagg[scen][0],
                       'S': dictagg[scen][1].sum(axis=0)[:-1],
                       'E': dictagg[scen][2].sum(axis=0)[:-1],
                       'I': dictagg[scen][3].sum(axis=0)[:-1],
                       'R': dictagg[scen][4].sum(axis=0)[:-1]})
df['Populácia'] = N
df['E_cum'] =  (N - df['S'])
df['I_cum'] = (N - df['S'] - df['E'])
new_infect = np.array(df['I_cum'].iloc[1:]) - np.array(df['I_cum'].iloc[0:-1]) 
new_infect = np.insert(new_infect, 0, first_infections.sum() , axis = 0)
new_recovered = np.array( df['R'].iloc[1:]) -np.array(df['R'].iloc[0:-1]) 
new_recovered = np.insert(new_recovered, 0, gamma*first_infections.sum() , axis = 0)
df['Nové infekcie'] = new_infect
df['Novovyliečení'] = new_recovered
df.columns = ['Dni','Náchylní','Exponovaní','Infikovaní','Vyliečení', 'Populácia','Niekedy exponovaní','Niekedy infikovaní',
              'Nové infekcie', 'Novovyliečení']
df.to_excel('./tabulky/v3_19.4.2020SEIR_vystup_Default_R0_0.8_MRK_zapnute.xlsx', index = False)

In [ ]:
# Graf pre pocet novych infekcii
scen_vyber = [0,1,2]
D = pd.DataFrame({'Dni':np.arange(0, total_days+1),
                 'S_high': (dictagg['SIR_scenar' + str(scen_vyber[0])].sus)*N,
                 'S_med': (dictagg['SIR_scenar' + str(scen_vyber[1])].sus)*N,
                 'S_low': (dictagg['SIR_scenar' + str(scen_vyber[2])].sus)*N })

N = pop.popul.sum()
D['I_cum_high'] = N - D['S_high']
D['I_cum_med'] = N -  D['S_med']
D['I_cum_low'] = N -  D['S_low']

D = D.drop(['S_high', 'S_med', 'S_low'], axis = 1)

## Graf poctu novych nakaz v case
new_inf_high = np.array(D['I_cum_high'].iloc[1:]) - np.array(D['I_cum_high'].iloc[0:-1])
new_inf_med  = np.array(D['I_cum_med'].iloc[1:]) - np.array(D['I_cum_med'].iloc[0:-1])
new_inf_low  = np.array(D['I_cum_low'].iloc[1:]) - np.array(D['I_cum_low'].iloc[0:-1])

E = pd.DataFrame({'Deň':np.arange(1, total_days+1), 
                  'Nové infekcie, R0 1.10': new_inf_high, 'Nové infekcie, R0 0.80':new_inf_med, 
                  'Nové infekcie, R0 0.80 socio-demog. neutral':new_inf_low})

sns.set(rc={'figure.figsize':(11, 4)})    

months = mdates.MonthLocator()

day_zero = dt.datetime.now() - dt.timedelta(days = 2) 
end = day_zero + dt.timedelta(days=total_days)
days = mdates.drange(day_zero,end,dt.timedelta(days=1))

fig, ax = plt.subplots()
clrs=['red','orange','green','blue']
with sns.axes_style("darkgrid"):
    epochs = list( E['Deň'] )    
    for i in range(3):
        means = np.array(E.iloc[:, i+1] , dtype=np.float64)
        ax.plot(days, means, c=clrs[i])
        #ax.set_xlim([dt.date(2020, 3, 22), dt.date(2021, 1, 1)])
        ax.xaxis.set_major_locator(months)
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%b' '%y'))
plt.xticks(rotation=45)
plt.title('Počet nových infekcií')
plt.legend(['Scenár1: R0 1.10', 'Scenár2: R0 0.80', 'Scenár3: R0 0.80, socio-demog. neutral'])
plt.tight_layout()
#plt.ylabel('Počet')

path = './plots/' + verzia_modelu + '_' + 'Inf_nove' + '_' + datum_cas + '.png'
plt.savefig(path, dpi=300)
plt.close

### Export na oblasti+MRK

In [ ]:
with open(path_hlavne,'rb') as f:
    dictagg=pickle.load(f)
    f.close()

In [ ]:
n_regions = len(merges)
n_munic = pop.shape[0]
n_mrk = OD_merged.shape[0] - n_regions
n_regions, n_mrk
df_microregions = pd.read_excel("./zdroje int/microregions.xlsx", 1)                                                                    
df_microregions                                                                                           
regions = defaultdict(list)                                                                               
munic_to_region = {}                                                                                      
for x in df_microregions.itertuples():                                                                    
    try:                                                                                                  
        munic = int(x[8])                                                                                 
    except ValueError:   # tu su nejake mestske casti                                                     
        continue                                                                                          
    region = x[1]                                                                                         
    regions[region].append(munic)                                                                                                                                       
    munic_to_region[munic] = region
names = [[] for _ in range(OD_merged.shape[0])]
for i in range(n_munic):
    munic = pop.munic[i]
    if munic in munic_to_region:
        region = munic_to_region[munic]    
        names[new_ids[i]].append(region)
for i in range(n_mrk):
    names[new_ids[n_munic + i]].append("MRK %s" % (df_coords["NM4"][base_munic[i+n_munic]]))
for n in names:
    if len(n) > 1:
        for n2 in n:
            assert n2 == n[0]
names = [x[0] for x in names]

In [ ]:
a = pd.DataFrame(np.transpose(dictagg['obce_scenar1'][3])) 
a.columns = names
a = a.iloc[0:600]
dni = np.arange(1,total_days+1)
a.insert(0, 'Deň', dni)
path = './tabulky/' + verzia_modelu + '_' + 'Vystup_regiony_a_MRK' + '_' + datum_cas + '.xlsx'
a.to_excel(path, index = False)